In [219]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# 날씨 데이터 불러오기

In [220]:
# whether csv read, 일시를 index로 넣고 datetime 형태로 변환함
whether = pd.read_csv('./data/Whether.csv', encoding='euc-kr', parse_dates=['일시'], index_col='일시')
# 강수량, 적설이 null인 경우 모두 0으로 바꿈
whether = whether.fillna(0)
# 일시의 freq을 hour로 변경
whether.index.freq = 'h'

'''
데이터가 누락된 시간을 확인하기 위한 부분 (파일로 받은 경우)
API로 받는 경우 날짜는 있지만 모든 데이터가 Null값이기 때문에 해당 조건으로 수정해야 할 것으로 보임

j = whether.index[0]
for i in whether.index:
    if i != j + pd.Timedelta("1 hour"):
        print(j, i)
    j = i
'''
# 지점코드, 지점명을 없앰
whether = whether.iloc[:, [2,3,4,5,6]]
whether

,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
일시,,,,,
2020-10-01 00:00:00,17,0,0,92,0
2020-10-01 01:00:00,17,0,0,94,0
2020-10-01 02:00:00,17,0,1,93,0
2020-10-01 03:00:00,17,0,0,94,0
2020-10-01 04:00:00,16,0,0,95,0
...,...,...,...,...,...
2022-10-01 19:00:00,21,0,0,54,0
2022-10-01 20:00:00,20,0,1,63,0
2022-10-01 21:00:00,19,0,1,73,0


# 차량 데이터 불러오기 및 전처리

# 한맥

장비별, 일시별 구분  
입차 대수 - 출차 대수 = 현재 차량 대수  
한맥의 경우 FilePath로부터 입출차 시간 추출  
정기권 등 구분 X

In [221]:
def pretreatment(df):
    df['일시'] = pd.to_datetime(df['FilePath'].apply(lambda x: x[12:26]), format='%Y%m%d%H%M%S')
    df['일시'] = pd.to_datetime(df['일시']).apply(del_min_sec)
    df = df[['일시', 'FilePath', 'InOutDiv']]
    df = df.rename({'FilePath':'주차일련번호'}, axis=1)
    return df

def del_min_sec(x):
    return x.replace(minute=0, second=0)

car_normal = pd.read_csv('./data/일반차량.csv')
car_normal = pretreatment(car_normal)

car_ticket = pd.read_csv('./data/정기권차량.csv')
car_ticket = pretreatment(car_ticket)




car_ticket_in = car_ticket[car_ticket['InOutDiv']==1].iloc[:,:2]


car_ticket_in = car_ticket_in.groupby('일시').count().reset_index().rename({'주차일련번호':'입차대수'}, axis=1)

car_ticket_out = car_ticket[car_ticket['InOutDiv']==2].iloc[:,:2]
car_ticket_out = car_ticket_out.groupby('일시').count().reset_index().rename({'주차일련번호':'출차대수'}, axis=1)

car_ticket_all = pd.merge(car_ticket_in, car_ticket_out, on='일시', how='outer').fillna(0).sort_values('일시').reset_index(drop=True)
car_ticket_all['입출차 차이'] = car_ticket_all['입차대수'] - car_ticket_all['출차대수']
car_ticket_all['차량구분'] = '정기권'
car_ticket_all['주차장'] = '주차1동, 주차2동'


# 일반
car_normal_in = car_normal[car_normal['InOutDiv']==1].iloc[:,:2]
car_normal_in = car_normal_in.groupby('일시').count().reset_index().rename({'주차일련번호':'입차대수'}, axis=1)

car_normal_out = car_normal[car_normal['InOutDiv']==2].iloc[:,:2]
car_normal_out = car_normal_out.groupby('일시').count().reset_index().rename({'주차일련번호':'출차대수'}, axis=1)

car_normal_all = pd.merge(car_normal_in, car_normal_out, on='일시', how='outer').fillna(0).sort_values('일시').reset_index(drop=True)
car_normal_all['입출차 차이'] = car_normal_all['입차대수'] - car_normal_all['출차대수']
car_normal_all['차량구분'] = '비정기권'
car_normal_all['주차장'] = '주차1동, 주차2동'

In [259]:
car_ticket[car_ticket['InOutDiv']==1].iloc[:,:2].sort_values('일시')

,일시,주차일련번호
42611,2020-10-01 08:00:00,\2020\10\01\2020100108312236조5492F.jpg
42609,2020-10-01 14:00:00,\2020\10\01\2020100114382068우6620F.jpg
42606,2020-10-01 18:00:00,\2020\10\01\2020100118235921어9903F.jpg
42604,2020-10-02 08:00:00,\2020\10\02\2020100208551836조5492F.jpg
42603,2020-10-02 09:00:00,\2020\10\02\2020100209563607오8177F.jpg
...,...,...
10,2022-10-01 17:00:00,\2022\10\01\2022100117134120노0407F.jpg
7,2022-10-01 18:00:00,\2022\10\01\20221001184421267구1366F.jpg
6,2022-10-01 18:00:00,\2022\10\01\2022100118505060모8494F.jpg
4,2022-10-01 19:00:00,\2022\10\01\20221001192516104허2500F.jpg


# 다래

In [2]:
import pandas as pd

In [3]:
darae_in = pd.read_csv('./data/VW_InCarinfo (차량 입차 정보 조회).csv')
darae_in = darae_in.dropna()
darae_out = pd.read_csv('./data/VW_OutCarinfo (차량 출차 정보 조회).csv')

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_84104/2210927378.py:1: DtypeWarning: Columns (0,1,2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  darae_in = pd.read_csv('./data/VW_InCarinfo (차량 입차 정보 조회).csv')


In [11]:
darae_out[darae_out['출차고객구분'] != '정기권']

,출차일자,출차시각,차량번호,주차일련번호,출차카드번호,장비명,출차구분,출차고객구분,출차여부,차량종류
0,2022-09-01,05:32:24,94고6836,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
1,2022-09-01,05:52:37,94고6836,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
2,2022-09-01,06:12:11,94고6836,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
3,2022-09-01,06:18:27,82도4072,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
4,2022-09-01,06:39:34,94고6836,1.111422e+16,1.000000e+14,13-3동출구#1,정기권출차,정기권,출차,소형
...,...,...,...,...,...,...,...,...,...,...
38667,2022-10-05,17:40:15,71하3305,1.411220e+16,1.000000e+14,15동지하입구,정상,정기권,미출차,소형
38668,2022-10-05,19:03:47,50머4665,1.411220e+16,1.000000e+14,15동지하입구,정상,정기권,미출차,소형
38669,2022-10-05,19:44:02,323러9283,1.411220e+16,1.000000e+14,15동지하입구,정상,정기권,미출차,소형
38670,2022-10-05,19:46:40,09도7906,1.411220e+16,1.000000e+14,15동지하입구,정상,정기권,미출차,소형


In [13]:
darae_in['입차고객구분'].value_counts()

정기권                   38616
일반                       53
할인                        3
Name: 입차고객구분, dtype: int64

In [222]:
darae_in = pd.read_csv('./data/VW_InCarinfo (차량 입차 정보 조회).csv')
darae_in = darae_in.dropna()
darae_out = pd.read_csv('./data/VW_OutCarinfo (차량 출차 정보 조회).csv')
darae_in[['입차고객구분']]= darae_in[['입차고객구분']].apply(lambda x: x.str.strip(), axis = 1)
darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']] = darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)
darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']]= darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_40230/2057935467.py:1: DtypeWarning: Columns (0,1,2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  darae_in = pd.read_csv('./data/VW_InCarinfo (차량 입차 정보 조회).csv')


## 이름 바꾸기 13_1동 지하 1 > 13_1동

In [223]:
import re
def slicing(df,i) :
    pattern = '.*동'
    reg = re.search(pattern, df['장비명'][i])
    reg = reg.group()
    df['주차장'][i] = reg
    return reg

darae_in['주차장'] = 0
for i in range(len(darae_in)):
    slicing(darae_in,i)
darae_out['주차장'] = 0
for i in range(len(darae_out)):
    slicing(darae_out,i)

def change(x):
    if x =='13-1동' or x =='13-2동' or x =='13-3동':
        return '13동'
    elif x == '14-1동' or x == '14-2동':
        return '14동'
    else :
        return x
darae_in['주차장'] = darae_in['주차장'].apply(change)
darae_out['주차장'] = darae_out['주차장'].apply(change)

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_40230/3206655803.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_40230/3206655803.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg


- 입차 중복 > 입차기록 있으나 출차 기록 없는 경우
- 출차 중복 > 입차기록 없으나 출차 기록 있는 경우

In [224]:
darae_in['입차시간'] = darae_in['입차일자'] +  darae_in['입차시각']
darae_out['출차시간'] = darae_out['출차일자']  + darae_out['출차시각']
darae_in['입차시간'] = pd.to_datetime(darae_in['입차시간']).apply(del_min_sec)
darae_out['출차시간'] = pd.to_datetime(darae_out['출차시간']).apply(del_min_sec)

darae_in_ticket = darae_in[darae_in['입차고객구분']=='정기권']
darae_in_normal = darae_in[darae_in['입차고객구분']!='정기권']

darae_out_ticket = darae_out[darae_out['출차고객구분']=='정기권']
darae_out_normal = darae_out[darae_out['출차고객구분']!='정기권']

In [225]:
darae_in_ticket = darae_in_ticket.groupby(['입차시간','주차장']).count().reset_index().rename({'입차일자':'입차대수'}, axis=1).iloc[:,:3]
darae_in_ticket['고객구분'] = '정기권'
darae_in_normal = darae_in_normal.groupby(['입차시간','주차장']).count().reset_index().rename({'입차일자':'입차대수'}, axis=1).iloc[:,:3]
darae_in_normal['고객구분'] = '비정기권'

darae_out_ticket = darae_out_ticket.groupby(['출차시간','주차장']).count().reset_index().rename({'출차일자':'출차대수'}, axis=1).iloc[:,:3]
darae_out_ticket['고객구분'] = '정기권'
darae_out_normal = darae_out_normal.groupby(['출차시간','주차장']).count().reset_index().rename({'출차일자':'출차대수'}, axis=1).iloc[:,:3]
darae_out_normal['고객구분'] = '비정기권'


## 빈 시간 채우기
- 처음에 9월 1일 0시 넣고 마지막에 9월 31일 23시 넣기

In [226]:
list(darae_out['주차장'].unique())

['13동', '15동', '12동', '14동']

In [227]:
def null_time(df,parking_name,time_column,InOut_column,start_time,end_time):
    df = df[df['주차장'] == parking_name]
    set_time = pd.date_range(start_time, end_time, freq='h')
    data =  pd.DataFrame(index=set_time)
    data = pd.merge(data, df, left_index=True, right_on=time_column, how='left')
    data['주차장'] = data['주차장'].fillna(parking_name)
    data[InOut_column] = data[InOut_column].fillna(0)
    return data

In [228]:
parking_list = list(darae_in['주차장'].unique())

darae_ticket_analysis = pd.DataFrame()
for i in parking_list:
    tmp_out = null_time(darae_out_ticket,i,'출차시간','출차대수','20220901000000','20220930235959').set_index('출차시간')
    tmp_in = null_time(darae_in_ticket,i,'입차시간','입차대수','20220901000000','20220930235959').set_index('입차시간')
    tmp_out = tmp_out.reset_index()
    tmp_in = tmp_in.reset_index()
    darae_ticket = pd.merge(tmp_in,tmp_out, left_on =['입차시간','주차장'], right_on = ['출차시간','주차장'] ,how = 'left')
    darae_ticket = darae_ticket.drop(['출차시간'],axis=1)
    darae_ticket = darae_ticket.rename({'입차시간' : '일시'},axis=1)
    darae_ticket['입출차 차이'] = darae_ticket['입차대수'] - darae_ticket['출차대수']
    darae_ticket['입출차 누계'] = darae_ticket['입출차 차이'].cumsum()
    darae_ticket = darae_ticket.set_index('일시')
    if darae_ticket['입출차 누계'].min() < 0:
        darae_ticket['입출차 누계'] -= darae_ticket['입출차 누계'].min()
    darae_ticket['연'] = darae_ticket.index.year
    darae_ticket['월'] = darae_ticket.index.month
    darae_ticket['일'] = darae_ticket.index.day
    darae_ticket['시각'] = darae_ticket.index.hour

    darae_ticket_analysis = pd.concat([darae_ticket_analysis,darae_ticket],axis=0)

darae_ticket_analysis = darae_ticket_analysis.rename({'고객구분_x' : '고객구분'},axis=1)
darae_ticket_analysis.drop('고객구분_y',axis=1,inplace =True)
darae_ticket_analysis['고객구분'] = darae_ticket_analysis['고객구분'].fillna('정기권')
darae_ticket_analysis


parking_list = list(darae_out['주차장'].unique())

darae_normal_analysis = pd.DataFrame()
for i in parking_list:
    tmp_out = null_time(darae_out_normal,i,'출차시간','출차대수','20220901000000','20220930235959').set_index('출차시간')
    tmp_in = null_time(darae_in_normal,i,'입차시간','입차대수','20220901000000','20220930235959').set_index('입차시간')
    tmp_out = tmp_out.reset_index()
    tmp_in = tmp_in.reset_index()
    darae_normal = pd.merge(tmp_in,tmp_out, left_on =['입차시간','주차장'], right_on = ['출차시간','주차장'] ,how = 'left')
    darae_normal = darae_normal.drop(['출차시간'],axis=1)
    darae_normal = darae_normal.rename({'입차시간' : '일시'},axis=1)
    darae_normal['입출차 차이'] = darae_normal['입차대수'] - darae_normal['출차대수']
    darae_normal['입출차 누계'] = darae_normal['입출차 차이'].cumsum()
    darae_normal = darae_normal.set_index('일시')
    if darae_normal['입출차 누계'].min() < 0:
        darae_normal['입출차 누계'] -= darae_normal['입출차 누계'].min()
    darae_normal['연'] = darae_normal.index.year
    darae_normal['월'] = darae_normal.index.month
    darae_normal['일'] = darae_normal.index.day
    darae_normal['시각'] = darae_normal.index.hour

    darae_normal_analysis = pd.concat([darae_normal_analysis,darae_normal],axis=0)

darae_normal_analysis = darae_normal_analysis.rename({'고객구분_x' : '고객구분'},axis=1)
darae_normal_analysis.drop('고객구분_y',axis=1,inplace =True)
darae_normal_analysis['고객구분'] = darae_normal_analysis['고객구분'].fillna('비정기권')

In [229]:
darae_analysis = pd.concat([darae_ticket_analysis,darae_normal_analysis])
darae_analysis

,주차장,입차대수,고객구분,출차대수,입출차 차이,입출차 누계,연,월,일,시각
일시,,,,,,,,,,
2022-09-01 00:00:00,13동,0,정기권,0,0,0,2022,9,1,0
2022-09-01 01:00:00,13동,0,정기권,0,0,0,2022,9,1,1
2022-09-01 02:00:00,13동,0,정기권,0,0,0,2022,9,1,2
2022-09-01 03:00:00,13동,2,정기권,0,2,2,2022,9,1,3
2022-09-01 04:00:00,13동,0,정기권,0,0,2,2022,9,1,4
...,...,...,...,...,...,...,...,...,...,...
2022-09-30 19:00:00,14동,0,비정기권,0,0,0,2022,9,30,19
2022-09-30 20:00:00,14동,0,비정기권,0,0,0,2022,9,30,20
2022-09-30 21:00:00,14동,0,비정기권,0,0,0,2022,9,30,21


In [230]:
from datetime import datetime

set_time = pd.date_range('20201001', '20221001230000', freq='h')
hanmeak_ticket_analysis = pd.DataFrame(index=set_time)
hanmeak_ticket_analysis = pd.merge(hanmeak_ticket_analysis, car_ticket_all, left_index=True, right_on='일시', how='left').fillna(0)
hanmeak_ticket_analysis = hanmeak_ticket_analysis.set_index('일시')
hanmeak_ticket_analysis['입출차 누계'] = hanmeak_ticket_analysis['입출차 차이'].cumsum()

# 누계가 음수인 경우, 최솟값만큼 기존에 차가 있었다고 가정함
if hanmeak_ticket_analysis['입출차 누계'].min() < 0:
    hanmeak_ticket_analysis['입출차 누계'] -= hanmeak_ticket_analysis['입출차 누계'].min()

hanmeak_ticket_analysis['연'] = hanmeak_ticket_analysis.index.year
hanmeak_ticket_analysis['월'] = hanmeak_ticket_analysis.index.month
hanmeak_ticket_analysis['일'] = hanmeak_ticket_analysis.index.day
hanmeak_ticket_analysis['시각'] = hanmeak_ticket_analysis.index.hour
hanmeak_ticket_analysis['주차장'] = hanmeak_ticket_analysis['주차장'].replace(0,'주차1동, 주차2동')
hanmeak_ticket_analysis['차량구분'] = hanmeak_ticket_analysis['차량구분'].replace(0,'정기권')

hanmeak_ticket_analysis = hanmeak_ticket_analysis[['주차장', '입차대수', '출차대수', '입출차 차이', '입출차 누계','차량구분', '연', '월', '일', '시각']]

,주차장,입차대수,출차대수,입출차 차이,입출차 누계,차량구분,연,월,일,시각
일시,,,,,,,,,,
2020-10-01 00:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,0
2020-10-01 01:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,1
2020-10-01 02:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,2
2020-10-01 03:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,3
2020-10-01 04:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,4
...,...,...,...,...,...,...,...,...,...,...
2022-10-01 19:00:00,"주차1동, 주차2동",1,1,0,12,정기권,2022,10,1,19
2022-10-01 20:00:00,"주차1동, 주차2동",0,1,-1,11,정기권,2022,10,1,20
2022-10-01 21:00:00,"주차1동, 주차2동",0,0,0,11,정기권,2022,10,1,21


In [236]:
from datetime import datetime

set_time = pd.date_range('20201001', '20221001230000', freq='h')
hanmeak_normal_analysis = pd.DataFrame(index=set_time)
hanmeak_normal_analysis = pd.merge(hanmeak_normal_analysis, car_normal_all, left_index=True, right_on='일시', how='left').fillna(0)
hanmeak_normal_analysis = hanmeak_normal_analysis.set_index('일시')
hanmeak_normal_analysis['입출차 누계'] = hanmeak_normal_analysis['입출차 차이'].cumsum()
# 누계가 음수인 경우, 최솟값만큼 기존에 차가 있었다고 가정함
if hanmeak_normal_analysis['입출차 누계'].min() < 0:
    hanmeak_normal_analysis['입출차 누계'] -= hanmeak_normal_analysis['입출차 누계'].min()

hanmeak_normal_analysis['연'] = hanmeak_normal_analysis.index.year
hanmeak_normal_analysis['월'] = hanmeak_normal_analysis.index.month
hanmeak_normal_analysis['일'] = hanmeak_normal_analysis.index.day
hanmeak_normal_analysis['시각'] = hanmeak_normal_analysis.index.hour
hanmeak_normal_analysis['주차장'] = hanmeak_normal_analysis['주차장'].replace(0,'주차1동, 주차2동')
hanmeak_normal_analysis['차량구분'] = hanmeak_normal_analysis['차량구분'].replace(0,'비정기권')

hanmeak_normal_analysis = hanmeak_normal_analysis[['주차장', '입차대수', '출차대수', '입출차 차이', '입출차 누계','차량구분', '연', '월', '일', '시각']]

In [240]:
hanmeak_ticket_analysis

,주차장,입차대수,출차대수,입출차 차이,입출차 누계,차량구분,연,월,일,시각
일시,,,,,,,,,,
2020-10-01 00:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,0
2020-10-01 01:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,1
2020-10-01 02:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,2
2020-10-01 03:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,3
2020-10-01 04:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,4
...,...,...,...,...,...,...,...,...,...,...
2022-10-01 19:00:00,"주차1동, 주차2동",1,1,0,12,정기권,2022,10,1,19
2022-10-01 20:00:00,"주차1동, 주차2동",0,1,-1,11,정기권,2022,10,1,20
2022-10-01 21:00:00,"주차1동, 주차2동",0,0,0,11,정기권,2022,10,1,21


In [250]:
hanmeak_ticket_analysis

,주차장,입차대수,출차대수,입출차 차이,입출차 누계,차량구분,연,월,일,시각
일시,,,,,,,,,,
2020-10-01 00:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,0
2020-10-01 01:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,1
2020-10-01 02:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,2
2020-10-01 03:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,3
2020-10-01 04:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,4
...,...,...,...,...,...,...,...,...,...,...
2022-10-01 19:00:00,"주차1동, 주차2동",1,1,0,12,정기권,2022,10,1,19
2022-10-01 20:00:00,"주차1동, 주차2동",0,1,-1,11,정기권,2022,10,1,20
2022-10-01 21:00:00,"주차1동, 주차2동",0,0,0,11,정기권,2022,10,1,21


In [251]:
darae_ticket_analysis

,주차장,입차대수,고객구분,출차대수,입출차 차이,입출차 누계,연,월,일,시각
일시,,,,,,,,,,
2022-09-01 00:00:00,13동,0,정기권,0,0,0,2022,9,1,0
2022-09-01 01:00:00,13동,0,정기권,0,0,0,2022,9,1,1
2022-09-01 02:00:00,13동,0,정기권,0,0,0,2022,9,1,2
2022-09-01 03:00:00,13동,2,정기권,0,2,2,2022,9,1,3
2022-09-01 04:00:00,13동,0,정기권,0,0,2,2022,9,1,4
...,...,...,...,...,...,...,...,...,...,...
2022-09-30 19:00:00,15동,1,정기권,20,-19,61,2022,9,30,19
2022-09-30 20:00:00,15동,0,정기권,6,-6,55,2022,9,30,20
2022-09-30 21:00:00,15동,0,정기권,0,0,55,2022,9,30,21


In [253]:
data_ticket = pd.concat([hanmeak_ticket_analysis,darae_ticket_analysis]).sort_index()
data_ticket = pd.merge(data_ticket, whether, left_index=True, right_index=True, how='left')
data_ticket.drop('고객구분',axis=1,inplace = True)

data_normal = pd.concat([hanmeak_normal_analysis,darae_normal_analysis]).sort_index()
data_normal = pd.merge(data_normal, whether, left_index=True, right_index=True, how='left')


In [255]:

data_ticket.to_csv('주차예측(한맥+다래)_정기권.csv')
data_normal.to_csv('주차예측(한맥+다래)_비정기권.csv')

In [254]:
data_ticket

,주차장,입차대수,출차대수,입출차 차이,입출차 누계,차량구분,연,월,일,시각,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
일시,,,,,,,,,,,,,,,
2020-10-01 00:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,0,17,0,0,92,0
2020-10-01 01:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,1,17,0,0,94,0
2020-10-01 02:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,2,17,0,1,93,0
2020-10-01 03:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,3,17,0,0,94,0
2020-10-01 04:00:00,"주차1동, 주차2동",0,0,0,96,정기권,2020,10,1,4,16,0,0,95,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-01 19:00:00,"주차1동, 주차2동",1,1,0,12,정기권,2022,10,1,19,21,0,0,54,0
2022-10-01 20:00:00,"주차1동, 주차2동",0,1,-1,11,정기권,2022,10,1,20,20,0,1,63,0
2022-10-01 21:00:00,"주차1동, 주차2동",0,0,0,11,정기권,2022,10,1,21,19,0,1,73,0
